 # Using met.no forecasts with weather-tools



 This notebook-style script shows how to fetch met.no forecasts and combine them

 with SILO observations using the new `weather_tools` APIs. Open it directly in

 JupyterLab or VS Code with Jupytext to run the cells interactively.

 ## Prerequisites

 - Network access to `https://api.met.no` and a valid contact e-mail for the User-Agent header.

 - Set `SILO_API_KEY` (your registered SILO e-mail) or pass `api_key=` directly to `SiloAPI`.

 - Install `weather-tools` in your environment (either via `pip install weather-tools` or the local checkout).

 - Optional: enable logging (e.g. `logging.basicConfig(level=logging.INFO)`) to inspect requests.

In [1]:
import pandas as pd

from weather_tools.merge_weather_data import get_merge_summary, merge_historical_and_forecast
from weather_tools.metno_api import MetNoAPI
from weather_tools.metno_models import MetNoFormat, MetNoQuery
from weather_tools.silo_api import SiloAPI
from weather_tools.silo_models import AustralianCoordinates


 ## 1. Configure the met.no API client

 met.no requires a descriptive User-Agent string that includes contact details.

 Replace the placeholder below with your own application name and e-mail.

In [2]:
api = MetNoAPI(
    # user_agent="weather-tools-example/0.1 (contact: you@example.com)",
    enable_cache=True,
    debug=True,
)


 ## 2. Build a forecast query for your location

 The `AustralianCoordinates` model validates that the latitude and longitude sit

 within the range supported by SILO (GDA94 datum). Adjust the coordinates to

 target your site of interest.

In [3]:
coordinates = AustralianCoordinates(latitude=-33.86, longitude=151.21)  # Sydney, NSW
query = MetNoQuery(coordinates=coordinates, format=MetNoFormat.COMPACT)

response = api.query_forecast(query)
metadata = response.get_meta()

metadata


🌐 Constructed URL: https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21
📋 User-Agent: weather-tools/0.0.0 (Python 3.12)


{'updated_at': '2025-10-22T13:15:59Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'cloud_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

 ## 3. Convert forecasts to daily and hourly tables

 `MetNoAPI.to_dataframe` can aggregate the hourly GeoJSON payload into the daily

 schema (`DailyWeatherSummary`) or expose the raw hourly data by disabling the

 aggregation flag.

In [4]:
daily_forecasts = api.to_dataframe(response, aggregate_to_daily=True)
hourly_forecasts = api.to_dataframe(response, aggregate_to_daily=False)

daily_forecasts.head()


,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-22,16.6,23.0,NaN,6.881818,8.4,43.727273,1010.463636,62.645455,partlycloudy_night
1,2025-10-23,15.7,21.3,NaN,5.020833,7.9,64.683333,1014.287500,46.745833,partlycloudy_day
2,2025-10-24,14.7,24.2,NaN,5.954167,11.7,71.870833,1015.658333,68.066667,partlycloudy_day
3,2025-10-25,16.1,22.7,NaN,6.822222,11.1,61.666667,1012.888889,56.077778,cloudy
4,2025-10-26,17.9,23.2,0.5,2.850000,3.5,66.300000,1000.575000,27.750000,partlycloudy_day


In [5]:
hourly_forecasts.head()


,time,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,next_1_hours_precipitation_amount,next_6_hours_precipitation_amount
0,2025-10-22 13:00:00+00:00,1005.4,23.0,83.6,34.8,277.3,7.5,0.0,NaN
1,2025-10-22 14:00:00+00:00,1005.3,22.1,64.8,36.2,287.5,6.8,0.0,NaN
2,2025-10-22 15:00:00+00:00,1007.4,21.1,71.9,39.0,214.8,5.3,0.0,NaN
3,2025-10-22 16:00:00+00:00,1008.3,19.8,70.3,40.9,209.2,6.8,0.0,NaN
4,2025-10-22 17:00:00+00:00,1009.7,19.0,68.7,42.1,211.4,8.3,0.0,NaN


 ### Quick helper: `get_daily_forecast`

 Prefer `get_daily_forecast` if you simply need the daily summaries as Pydantic

 models and want to control the number of days returned.

In [6]:
summaries = api.get_daily_forecast(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    days=5,
)

pd.DataFrame([summary.model_dump() for summary in summaries])


🌐 Constructed URL: https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21
📋 User-Agent: weather-tools/0.0.0 (Python 3.12)


,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-22,16.6,23.0,NaN,6.881818,8.4,43.727273,1010.463636,62.645455,partlycloudy_night
1,2025-10-23,15.7,21.3,NaN,5.020833,7.9,64.683333,1014.287500,46.745833,partlycloudy_day
2,2025-10-24,14.7,24.2,NaN,5.954167,11.7,71.870833,1015.658333,68.066667,partlycloudy_day
3,2025-10-25,16.1,22.7,NaN,6.822222,11.1,61.666667,1012.888889,56.077778,cloudy
4,2025-10-26,17.9,23.2,0.5,2.850000,3.5,66.300000,1000.575000,27.750000,partlycloudy_day


 ## 4. Merge met.no forecasts with SILO history

 Pull the last five days from the SILO DataDrill API for the same coordinates,

 then merge that history with the met.no forecast. The helper automatically

 converts column names, fills optional variables (when enabled), and annotates

 the data source for each record.

In [9]:
first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date - pd.Timedelta(days=1)
history_start = history_end - pd.Timedelta(days=4)

silo_api = SiloAPI()
raw_silo = silo_api.get_gridded_data(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    start_date=history_start.strftime("%Y%m%d"),
    end_date=history_end.strftime("%Y%m%d"),
    variables=["rainfall", "max_temp", "min_temp"],
)

silo_history = raw_silo.rename(columns={"YYYY-MM-DD": "date"}).copy()

silo_history["date"] = pd.to_datetime(silo_history["date"])

required_cols = ["date", "min_temp", "max_temp", "daily_rain"]
missing = [col for col in required_cols if col not in silo_history.columns]
if missing:
    raise ValueError(f"Missing required columns from SILO response: {missing}")

silo_data = silo_history[required_cols].sort_values("date").reset_index(drop=True)

merged = merge_historical_and_forecast(
    silo_data=silo_data,
    metno_data=daily_forecasts,
    fill_missing=True,
    overlap_strategy="prefer_silo",
)

merged


/Users/hfsi/Developer/weather_tools/src/weather_tools/merge_weather_data.py:146: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([silo_df, metno_df], ignore_index=True)


,min_temp,max_temp,vp,max_wind_speed,date,data_source,avg_wind_speed,forecast_generated_at,day,year,avg_cloud_fraction,mslp,daily_rain,is_forecast,dominant_weather_symbol
0,17.9,33.2,NaN,NaN,2025-10-17,silo,NaN,NaT,NaN,NaN,NaN,NaN,0.0,False,NaN
1,15.7,22.0,NaN,NaN,2025-10-18,silo,NaN,NaT,NaN,NaN,NaN,NaN,0.7,False,NaN
2,15.5,25.5,NaN,NaN,2025-10-19,silo,NaN,NaT,NaN,NaN,NaN,NaN,0.0,False,NaN
3,17.0,36.0,NaN,NaN,2025-10-20,silo,NaN,NaT,NaN,NaN,NaN,NaN,0.0,False,NaN
4,17.7,23.7,NaN,NaN,2025-10-21,silo,NaN,NaT,NaN,NaN,NaN,NaN,0.0,False,NaN
5,16.6,23.0,43.727273,8.4,2025-10-22,metno,6.881818,2025-10-22 13:54:20.052139+00:00,295.0,2025.0,62.645455,1010.463636,NaN,True,partlycloudy_night
6,15.7,21.3,64.683333,7.9,2025-10-23,metno,5.020833,2025-10-22 13:54:20.052139+00:00,296.0,2025.0,46.745833,1014.287500,NaN,True,partlycloudy_day
7,14.7,24.2,71.870833,11.7,2025-10-24,metno,5.954167,2025-10-22 13:54:20.052139+00:00,297.0,2025.0,68.066667,1015.658333,NaN,True,partlycloudy_day
8,16.1,22.7,61.666667,11.1,2025-10-25,metno,6.822222,2025-10-22 13:54:20.052139+00:00,298.0,2025.0,56.077778,1012.888889,NaN,True,cloudy
9,17.9,23.2,66.300000,3.5,2025-10-26,metno,2.850000,2025-10-22 13:54:20.052139+00:00,299.0,2025.0,27.750000,1000.575000,0.5,True,partlycloudy_day


In [8]:
merge_summary = get_merge_summary(merged)
merge_summary


{'total_records': 16,
 'silo_records': np.int64(5),
 'metno_records': np.int64(11),
 'date_range': {'start': Timestamp('2025-10-17 00:00:00'),
  'end': Timestamp('2025-11-01 00:00:00'),
  'days': 16},
 'silo_period': {'start': Timestamp('2025-10-17 00:00:00'),
  'end': Timestamp('2025-10-21 00:00:00')},
 'metno_period': {'start': Timestamp('2025-10-22 00:00:00'),
  'end': Timestamp('2025-11-01 00:00:00')},
 'transition_date': Timestamp('2025-10-22 00:00:00')}

 ## 5. Next steps

 - Persist the merged DataFrame (e.g. `merged.to_parquet("combined_weather.parquet")`) for downstream analysis.

 - Explore additional columns in `hourly_forecasts` (wind, humidity, cloud cover) or enrich the SILO history with more variables as needed.